In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import json
import pandas as pd
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting,GenerationConfig

In [ ]:
def generate(Prompt,system_prompt,generation_config,safety_settings ):
    vertexai.init(project="hclsw-gcp-srsch", location="us-central1")
    model = GenerativeModel(
        "gemini-1.5-flash-001",
        system_instruction=[system_prompt]
    )
    responses = model.generate_content(
        [Prompt],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )
    return responses
    # for response in responses:
    #     print(response.text, end="")

In [ ]:
def prompt_template_6(Jira_story):
    Prompt = f"""
    Please break down the following Jira Story into appropriate Subtasks based on its context and the goals outlined in the story. Each subtask should have a clear title and description, outlining the key actions and objectives. Ensure that the subtasks align with the parent story's context and overall goals.

    - **User Input:**
      - Jira Story Details: Includes Title, Summary, Context, and Acceptance Criteria.

    - **Expected Output:**
      - **Subtasks**: A structured breakdown of tasks with clear descriptions. Each subtask should focus on a specific action related to the parent story.

    - **Example Format:**

      **User Input:**
      - **Parent Story:**

        **Story 1: Generate Reports for Multiple Use Cases**

        - **Summary**: The system should be able to generate reports for various use cases.

        - **Context**: Users need to understand the results of the model across different scenarios. Reports provide a structured way to present this information.

        - **Acceptance Criteria**:
          - The system should support the generation of reports for at least three distinct use cases.
          - Each report should contain relevant information specific to the use case, such as data used, model parameters, and key findings.
          - Users should be able to select the use cases for which they want to generate reports.

      **Output:**

      - **Story: Generate Reports for Multiple Use Cases**

        - **Subtask 1: Identify Data and Parameters for Each Report**

          - **Description**: Identify the specific data and parameters needed for generating the reports for each use case.

        - **Subtask 2: Design Report Templates**

          - **Description**: Design templates for the reports that will display the relevant information.

        - **Subtask 3: Implement Report Generation Logic**

          - **Description**: Develop the logic needed to generate reports based on user selection.

        - **Subtask 4: Enable User Selection of Use Cases**

          - **Description**: Implement a user interface that allows users to select the use cases for which they want to generate reports.

    **Important Notes:**
    - Do **not** assign Story Points to the subtasks.
    - Do **not** analyze or recommend changes to the parent story’s Story Points.
    - Focus on breaking down the story into clear, actionable tasks.
    - Each subtask should be concise and aligned with the overall goal of the parent story.

    Now, based on the following Jira Story, generate the appropriate subtasks:

    **Parent Story: {Jira_story}**
    """

    system_prompt = """
    You will receive a Jira Story containing details such as Context, Acceptance Criteria, and Summary.

    Your task is to:

    - **Analyze the Story**: Understand the context, goals, and objectives of the story.
    - **Break Down into Subtasks**: Decompose the story into smaller, actionable tasks based on the given context.
    - **Provide Clear Subtask Descriptions**: Write each subtask description in a way that is easy to understand and actionable.

    **Output Requirements:**
    - Each subtask should have a **Title** and a **Description**.
    - Do **not** assign Story Points to the subtasks.
    - Do **not** analyze or suggest changes to the parent story’s Story Points.

    **Guidelines:**
    - Each subtask should focus on one specific action or goal within the context of the parent story.
    - Ensure that the subtasks are concise and actionable.
    - The subtasks must align with the overall objective and context of the parent story.
    """

    return Prompt, system_prompt


In [ ]:
def inference(Jira_story):

  prompt , system_prompt = prompt_template_6(Jira_story)

  generation_config = {
      "max_output_tokens": 1024,
      "temperature": 0.5,
      "top_p": 0.95,
      "seed":42
  }

  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
    ]

  result = generate(prompt,system_prompt,generation_config,safety_settings)

  return result.text



In [ ]:
def inference_2(Jira_story):

  prompt , system_prompt = prompt_template_6(Jira_story)

  generation_config = {
    "max_output_tokens": 1024,
    "temperature": 0.5,
    "top_p": 0.95,
    "response_mime_type": "application/json",
    "response_schema": {
        "type": "OBJECT",
        "properties": {
            "stories": {
                "type": "ARRAY",
                "items": {
                    "type": "OBJECT",
                    "properties": {
                        "story": {"type": "STRING"},
                        "subtasks": {
                            "type": "ARRAY",
                            "items": {
                                "type": "OBJECT",
                                "properties": {
                                    "subtask": {"type": "STRING"},
                                    "description": {"type": "STRING"}
                                }
                            }
                        }
                    }
                }
            }
        }
    },
    "seed": 42,
  }


  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
    ]

  result = generate(prompt,system_prompt,generation_config,safety_settings)

  return result.text



In [ ]:
# Function to format stories for inference
def format_stories(stories):
    formatted_list = []
    for story in stories:
        formatted_story = f"{story['story_number']}: {story['title']}\n\n"
        formatted_story += f"    - Summary: {story['summary']}\n\n"
        formatted_story += f"    - Context: {story['context']}\n\n"
        formatted_story += f"    - Acceptance Criteria:\n"

        for criteria in story["acceptance_criteria"]:
            formatted_story += f"      - {criteria}\n"

        formatted_story += f"\n    - Story Points: {story['story_points']}\n\n"

        formatted_list.append(formatted_story)

    return formatted_list

# Function to process uploaded JSON file
def process_json_file(file, start_idx, end_idx):
    df = pd.read_json(file.name)

    start_idx = max(0, min(start_idx, len(df) - 1))
    end_idx = max(start_idx, min(end_idx, len(df)))

    df = df.iloc[start_idx:end_idx]

    output_data = []  # Final output list to store processed results

    for _, row in df.iterrows():
        row_data = row.to_dict()  # Keep all original fields
        processed_data = []  # Reset processed data for each row

        try:
            data = json.loads(row['generated_stories'])  # Parse JSON from 'Answer' column
            formatted_stories = format_stories(data["stories"])  # Format each story

            # Apply inference_2 on each formatted story and store results
            for story in formatted_stories:
                generated_response = json.loads(inference_2(story))
                processed_data.append(generated_response)

        except json.JSONDecodeError:
            continue

        # Map processed data back to the original row
        row_data["Processed_Answers"] = processed_data
        output_data.append(row_data)

    # Save output to JSON file
    output_file = "processed_output.json"
    with open(output_file, "w") as f:
        json.dump(output_data, f, indent=4)

    return output_file

# Function to process direct story input using inference
def process_direct_input(story):
    generated_response = inference(story)  # Apply inference directly

    return generated_response  # Display output directly in UI






In [ ]:
# Gradio UI
def gradio_interface():
    with gr.Blocks() as demo:
        gr.Markdown("## JSON Story Processing To Subtask")

        with gr.Tab("Upload and Process JSON"):
            file_input = gr.File(label="Upload JSON File")
            start_idx = gr.Number(label="Start Index", value=0, interactive=True)
            end_idx = gr.Number(label="End Index", value=10, interactive=True)
            file_output = gr.File(label="Processed JSON Output", interactive=False)
            process_button = gr.Button("Process JSON")

            process_button.click(process_json_file, inputs=[file_input, start_idx, end_idx], outputs=file_output)

        with gr.Tab("Direct Story Processing"):
            story_input = gr.Textbox(label="Enter Story", lines=5, placeholder="Paste your story here")
            response_output = gr.Markdown(label="Generated Response")
            generate_button = gr.Button("Generate Response")

            generate_button.click(process_direct_input, inputs=[story_input], outputs=response_output)

    return demo

In [ ]:
# Launch the Gradio interface
gradio_interface().launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://650bcd42db9612a4fb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## experiments

In [ ]:
import pandas as pd

df = pd.read_json("/content/processed_output.json")

In [ ]:
import json

# Load JSON
data = json.loads(df['Answer'][0])

# Function to format stories and store in a list
def format_stories(stories):
    formatted_list = []
    for story in stories:
        formatted_story = f"{story['story_number']}: {story['title']}\n\n"
        formatted_story += f"    - Summary: {story['summary']}\n\n"
        formatted_story += f"    - Context: {story['context']}\n\n"
        formatted_story += f"    - Acceptance Criteria:\n"

        for criteria in story["acceptance_criteria"]:
            formatted_story += f"      - {criteria}\n"

        formatted_story += f"\n    - Story Points: {story['story_points']}\n\n"

        formatted_list.append(formatted_story)

    return formatted_list

# Store formatted stories in a list
formatted_stories_list = format_stories(data["stories"])

# Print each formatted story
for story in formatted_stories_list:
    print(story)


Story 1: Prompt Handling Interface

    - Summary: The system should provide a user interface for asking analysis questions.

    - Context: Users should be able to interact with the system through a dedicated interface to ask their analysis questions.

    - Acceptance Criteria:
      - The system should provide a user interface for entering analysis-related questions.
      - The interface should be intuitive and user-friendly.
      - The system should be able to handle various question formats and structures.
      - The system should be able to interpret and understand the user's intent from the question.
      - The system should be able to provide feedback on the quality of the question, such as clarity and completeness.
      - The system should be able to suggest alternative ways to phrase the question for better results.

    - Story Points: 5


Story 2: Prompt Optimization Recommendations

    - Summary: The system should provide recommendations on how to create optimal prom

In [ ]:
print(formatted_stories_list[0])

Story 1: Prompt Handling Interface

    - Summary: The system should provide a user interface for asking analysis questions.

    - Context: Users should be able to interact with the system through a dedicated interface to ask their analysis questions.

    - Acceptance Criteria:
      - The system should provide a user interface for entering analysis-related questions.
      - The interface should be intuitive and user-friendly.
      - The system should be able to handle various question formats and structures.
      - The system should be able to interpret and understand the user's intent from the question.
      - The system should be able to provide feedback on the quality of the question, such as clarity and completeness.
      - The system should be able to suggest alternative ways to phrase the question for better results.

    - Story Points: 5




In [ ]:
answer = inference_2(formatted_stories_list[1])

In [ ]:
print(answer)

{"stories": [{"story": "Story 2: Prompt Optimization Recommendations", "subtasks": [{"description": "Analyze user prompts to identify areas for improvement based on factors like clarity, specificity, and potential biases.", "subtask": "Analyze User Prompts"}, {"description": "Develop algorithms to generate recommendations for prompt optimization, including suggestions for rephrasing, adding context, specifying desired output format, and incorporating relevant keywords.", "subtask": "Develop Optimization Algorithms"}, {"description": "Design an interface to present recommendations to users in a clear and actionable manner, highlighting potential issues and providing specific suggestions for improvement.", "subtask": "Design Recommendation Interface"}, {"description": "Implement a feature that allows users to experiment with different prompt variations based on the provided recommendations and observe the impact on the results.", "subtask": "Implement Prompt Variation Experimentation"}]}

In [ ]:
print(answer)

## Story: Prompt Optimization Recommendations

### Subtasks:

**Subtask 1: Analyze Prompt Structure and Content**

- **Description:** Develop algorithms to analyze user-provided prompts, identifying key components like keywords, context, and structure. This analysis should identify potential areas for improvement based on best practices and common prompt optimization techniques.

**Subtask 2: Develop Recommendation Engine**

- **Description:** Implement a recommendation engine that analyzes the prompt analysis results and generates tailored suggestions for optimization. These recommendations should include specific changes to prompt wording, structure, or content, aimed at improving clarity, relevance, and effectiveness.

**Subtask 3: Generate Actionable Recommendations**

- **Description:** Ensure that the recommendations generated by the system are clear, concise, and actionable. They should be presented in a user-friendly format, providing specific instructions on how to modify the 

In [ ]:
print(answer)

## Story 2: Prompt Optimization Recommendations

### Subtasks:

**Subtask 1: Analyze User Prompts**

- Description: Develop a system that can analyze user-provided prompts to identify potential areas for improvement. This includes understanding the prompt's structure, identifying keywords, and analyzing the context of the question.

**Subtask 2: Define Optimization Criteria**

- Description: Define the criteria for evaluating prompt quality. This could include factors like clarity, relevance, specificity, and potential for bias.

**Subtask 3: Develop Recommendation Engine**

- Description: Develop an algorithm that can generate recommendations based on the analysis of the user's prompt and the defined optimization criteria. Recommendations should be tailored to the specific question and context.

**Subtask 4: Implement Recommendation Display**

- Description: Design and implement a user interface that presents the recommendations in a clear and concise manner. The interface should incl

In [ ]:
print(answer)

## Story: Prompt Handling Interface

### Subtasks:

**Subtask 1: Design User Interface for Question Input**

- Description: Design the user interface for entering analysis questions, ensuring it is intuitive and user-friendly. This includes elements like text boxes, buttons, and potentially a visual editor for formatting.

**Subtask 2: Implement Question Input Handling**

- Description: Implement the backend logic to receive and process user-entered questions. This includes handling different question formats, structures, and potential variations in phrasing.

**Subtask 3: Develop Question Interpretation Logic**

- Description: Develop the logic to interpret and understand the user's intent from the question. This might involve natural language processing (NLP) techniques to extract key concepts, entities, and relationships from the question.

**Subtask 4: Implement Question Quality Feedback**

- Description: Implement a mechanism to provide feedback to the user on the quality of their

In [ ]:
df['Answer'][0]

'{"stories": [{"acceptance_criteria": ["The system should provide a user interface for entering analysis-related questions.", "The interface should be intuitive and user-friendly.", "The system should be able to handle various question formats and structures.", "The system should be able to interpret and understand the user\'s intent from the question.", "The system should be able to provide feedback on the quality of the question, such as clarity and completeness.", "The system should be able to suggest alternative ways to phrase the question for better results."], "context": "Users should be able to interact with the system through a dedicated interface to ask their analysis questions.", "story_number": "Story 1", "story_points": "5", "summary": "The system should provide a user interface for asking analysis questions.", "title": "Prompt Handling Interface"}, {"acceptance_criteria": ["The system should analyze the user\'s prompt and identify potential areas for improvement.", "The sy

In [ ]:
import json
data = json.loads(df['Answer'][0])

In [ ]:
data['stories'][0]['story_number']

'Story 1'

In [ ]:
def prompt_template_5(Jira_story):
    Prompt = f"""
    Please break down the following Jira Story into appropriate Subtasks based on its context and the goals outlined in the story. Each subtask should have a clear description, outlining the key actions and objectives, based on the context of the story. Additionally, after breaking down the subtasks, please analyze if the total effort required seems to exceed the expected Story Points for the parent story. If the total effort feels higher than expected, provide a **recommendation** to adjust the Story Points for the parent story.

    - User Input:
       - Jira Story Details: Includes Title, Summary, Context, and Acceptance Criteria. Story Points for the parent story.

    - Expected Output:

      - Subtasks: A logical breakdown of tasks with clear descriptions. Each subtask should be aligned with the context of the parent story, ensuring that the subtasks follow the overall goals.

      - **Story Points Analysis**: After analyzing the parent story and its subtasks, evaluate if the total effort required seems higher than expected. If the total effort required exceeds the parent story's points, **recommend** an increase in the Story Points and suggest an appropriate Fibonacci number for the new value.

      - **Example Format**:

        User Input:

        - Parent Story:

          Story 1: Generate Reports for Multiple Use Cases

            - Summary: The system should be able to generate reports for various use cases.

            - Context: Users need to understand the results of the model across different scenarios. Reports provide a structured way to present this information.

            - Acceptance Criteria:

              - The system should support the generation of reports for at least three distinct use cases.

              - Each report should contain relevant information specific to the use case, such as data used, model parameters, and key findings.

              - Users should be able to select the use cases for which they want to generate reports.

            - Story Points: 5

        Output:

          - Story: Generate Reports for Multiple Use Cases

            - Subtask 1: Identify Data and Parameters for Each Report

              - Description: Identify the specific data and parameters needed for generating the reports for each use case.

            - Subtask 2: Design Report Templates

              - Description: Design templates for the reports that will display the relevant information.

            - Subtask 3: Implement Report Generation Logic

              - Description: Develop the logic needed to generate reports based on user selection.

            - Subtask 4: Enable User Selection of Use Cases

              - Description: Implement a user interface that allows users to select the use cases for which they want to generate reports.

            - Story Points Analysis: After analyzing the total effort required for the subtasks, it appears the work may exceed the originally allocated Story Points of 5. **Recommendation**: Increase the Story Points for the parent story to **8** to better reflect the expected effort and ensure the smooth execution of all subtasks.

      **Important Note:**
      - Do not assign Story Points to the subtasks in your response.
      - Focus on breaking down the story into clear, actionable tasks.
      - Provide a **recommendation** for adjusting the parent story's points only if you feel the total effort required exceeds the expected Story Points.

      Now, based on the context provided in the following Jira Story, generate the appropriate subtasks:

      Parent Story: {Jira_story}
    """

    system_prompt = """
    You will receive a Jira Story containing details such as Context, Acceptance Criteria, and Summary, along with the allocated Story Points for the parent story.

    Your task is to:

  - Analyze the Story: Fully understand the context, goals, and objectives of the story.
  - Break Down into Subtasks: Decompose the story into smaller, actionable tasks based on the given context. Ensure each subtask is aligned with the overall goals of the parent story.
  - Analyze Story Points: Evaluate if the total effort required for the subtasks seems higher than the expected Story Points for the parent story.
    - If the total effort for all subtasks exceeds the parent story’s points, **recommend** an increase in the parent story's Story Points, suggesting the appropriate Fibonacci number (e.g., 8).
  - Provide Clear Descriptions: Write each subtask description in a way that is easy to understand and actionable, considering the context provided.

  Output Requirements:
  For each subtask:

  - Provide a Title for the task.
  - Write a Description that explains the task's purpose and what needs to be done.
  - **Do not assign Story Points** for the subtasks in your response.
  - Only recommend a change to the parent story’s Story Points if the total effort for the subtasks seems to exceed the points allocated.

  Guidelines:
  - Each subtask should be focused on one specific action or goal within the context of the parent story.
  - Ensure that the subtasks are concise and actionable, making them clear for the team.
  - The subtasks must align with the overall objective and context of the parent story.

"""

    return Prompt, system_prompt
